In [36]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

df = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/data_for_models_train.pkl')


In [41]:
age_intervals = [0, 19, 30, 40, 60, 100]
age_labels = [0, 1, 2, 3, 4]
# Discretize the 'age' column into four classes
df['age_class'] = pd.cut(df['age'], bins=age_intervals, labels=age_labels, right=False).astype('str').tolist()
df['age_class'] = pd.to_numeric(df['age_class'])

# Initialize an empty DataFrame to store the sampled rows
sampled_df = pd.DataFrame()

# Define the number of rows to sample for each age_class
sample_size = 2

# Iterate over unique values in the 'age_class' column
for age_class in df['age_class'].unique():
    # Sample 'sample_size' rows for each age_class
    sampled_rows = df[df['age_class'] == age_class].sample(sample_size, random_state=1)
    # Append the sampled rows to the 'sampled_df'
    sampled_df = sampled_df.append(sampled_rows)

# Reset the index of the resulting DataFrame
sampled_df.reset_index(drop=True, inplace=True)

/data_local/slurm_job.10950149/ipykernel_15465/582124120.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_rows)
/data_local/slurm_job.10950149/ipykernel_15465/582124120.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_rows)
/data_local/slurm_job.10950149/ipykernel_15465/582124120.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_rows)
/data_local/slurm_job.10950149/ipykernel_15465/582124120.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_rows)
/data_local/slurm_job.10

In [43]:
# Shuffle data
sampled_df = sampled_df.sample(frac=1, random_state=40).reset_index(drop=True)
sampled_df

,user_id,is_male,age,masked_bio,long_text,age_class
0,1105503642268942337,True,70.0,LA GUERRA È UNA LEZIONE DI STORIA CHE I POPOLI...,Purtroppo al popolo italiano queste cose sembr...,4
1,825821903269003264,False,25.0,,#isola mandiamo Moreno come cavia per andare a...,1
2,458296542,False,14.0,Il mare era troppo salato e noi troppo graffiati.,ti amo.\n#MTVEMA where is The Weeknd????\nYOU ...,0
3,547976309,True,52.0,,Che bei colori!!! 😎😎😎\nQuello è il resto 🤪🤪🤪\n...,3
4,911968849167306753,False,25.0,Beatrice Caroline A. 🇺🇸🇮🇹 ItaloAmericana. Mila...,Ma esattamente cosa piace di Nikita? non sa ne...,1
5,525253183,False,32.0,io non bio.,Girerà a breve un video sul cyber-bullismo con...,2
6,417490284,True,46.0,,Ibra a quell'età per divertirsi.\nHo tentennat...,3
7,2924372112,False,30.0,🏳️‍🌈⚢ • ''Endure and survive'' • ⚽️🎮🎬 • #Killi...,In questa finale scontatissima di #Sanremo2023...,2
8,4439685929,True,10.0,,"grazie\nCiao sono Niccolò , vivo a none torino...",0
9,1530922237,False,60.0,,#fatela-staffetta-adamatrice\nTanto non vi vot...,4


In [44]:
import re

def twitter_features(
    df,
    include_bio=True,
    include_tweets=True,
    label_name='is_male',
    ):
    # check if there are any missing values (shouldn't be the case)
    if df.isnull().values.any():
        raise ValueError('The dataframe contains missing values')
    # Read each bio and tweets concatenation, splitting them by \n and
    # joining by '. ' if sentences don't already end with a dot, else join by ' '
    if include_bio:
        bios = df.masked_bio.apply(lambda x: [text + '.' if not (text.endswith('.') or text.endswith('!') or text.endswith('?') or text.endswith(';')) else text for text in x.split('\n')]).apply(lambda x: ' '.join(x)).apply(lambda x: re.sub('\r', '', x)).tolist()
    if include_tweets:
        tweets = df.long_text.apply(lambda x: [text + '.' if not (text.endswith('.') or text.endswith('!') or text.endswith('?') or text.endswith(';')) else text for text in x.split('\n')]).apply(lambda x: ' '.join(x)).apply(lambda x: re.sub('\r', '', x)).tolist()
    if include_bio and include_tweets:
        # Join each tweet and bio by 'Bio: ' and 'Tweets: '
        input_texts = ['Bio: ' + bio + '\n' + 'Tweets: ' + tweet for bio, tweet in zip(bios, tweets)]
    elif include_bio:
        input_texts = ['Bio: ' + bio for bio in bios]
    elif include_tweets:
        input_texts = ['Tweets: ' + tweet for tweet in tweets]

    # Read the gold labels
    if label_name == 'is_male':
        gold_labels = df[label_name].tolist()
        gold_labels = ['male' if label == True else 'female' for label in gold_labels]
    if label_name == 'age':
        gold_labels = df[label_name].astype(int).tolist()
    if label_name == 'age_interval':
        # define age classes
        age_intervals = [0, 19, 30, 40, 100]
        age_labels = [0, 1, 2, 3]
        # Discretize the 'age' column into four classes
        gold_labels = pd.cut(df['age'], bins=age_intervals, labels=age_labels, right=False).astype('str').tolist()

    return input_texts, gold_labels

input_texts, gold_labels = twitter_features(sampled_df)

In [45]:
shortened_texts = [' '.join(t.split()[:100]) for t in input_texts]

In [48]:
sampled_df['gender'] = sampled_df['is_male'].apply(lambda x: 'male' if x==True else 'female')

In [49]:
for i in range(len(sampled_df)):
    print(shortened_texts[i])
    print(f"Gender: {sampled_df.iloc[i]['gender']}")
    print(f"Age group: {sampled_df.iloc[i]['age_class']}")
    print()
    #print('-----------------------------------------------')


Bio: LA GUERRA È UNA LEZIONE DI STORIA CHE I POPOLI NON RICORDANO MAI ABBASTANZA. . Ma il primo a dimenticarla è stato l'autore. Tweets: Purtroppo al popolo italiano queste cose sembrano non riguardare. Troveranno il modo di non rispettare neanche questo richiamo.d5x. Chissà come mai il referendum sulla giustizia è argomento tabù per i notiziari e per i dibattiti televisivi. Ciò che sorprendente di più è vedere che esistono deficienti che sostengono le affermazioni di questa ignobile persona. L'Europa aspettava l'esito delle elezioni francesi per emettere le nuove sanzioni che prevederebbero lo stop al gas russo, di cui l'Italia
Gender: male
Age group: 4

Bio: . Tweets: #isola mandiamo Moreno come cavia per andare a visitare quei meravigliosi 7 nuovi pianeti a 40 anni luce da quiiiiiii! Malena non riesco proprio a definirti DONNA! Senza un minimo di contegno! #isola. e Sono Mamma.IL mondo del lavoro mi rifiuta. Spiegatemi come funziona l'ITALIA! @PaoloGentiloni @PolettiGiuliano. Mia fi

In [33]:
shortened_texts = ["This is a string with special characters: \u20AC and \u00A9", "Another string with special characters: \u2014 and \u2022"]

for text in shortened_texts:
    print(repr(text))

'This is a string with special characters: € and ©'
'Another string with special characters: — and •'


In [9]:
sampled_df

,user_id,is_male,age,masked_bio,long_text,age_class,gender
0,417490284,True,46.0,,Ibra a quell'età per divertirsi.\nHo tentennat...,3,male
1,547976309,True,52.0,,Che bei colori!!! 😎😎😎\nQuello è il resto 🤪🤪🤪\n...,3,male
2,911968849167306753,False,25.0,Beatrice Caroline A. 🇺🇸🇮🇹 ItaloAmericana. Mila...,Ma esattamente cosa piace di Nikita? non sa ne...,1,male
3,825821903269003264,False,25.0,,#isola mandiamo Moreno come cavia per andare a...,1,male
4,2924372112,False,30.0,🏳️‍🌈⚢ • ''Endure and survive'' • ⚽️🎮🎬 • #Killi...,In questa finale scontatissima di #Sanremo2023...,2,male
5,525253183,False,32.0,io non bio.,Girerà a breve un video sul cyber-bullismo con...,2,male
6,1105503642268942337,True,70.0,LA GUERRA È UNA LEZIONE DI STORIA CHE I POPOLI...,Purtroppo al popolo italiano queste cose sembr...,4,male
7,1530922237,False,60.0,,#fatela-staffetta-adamatrice\nTanto non vi vot...,4,male
8,4439685929,True,10.0,,"grazie\nCiao sono Niccolò , vivo a none torino...",0,male
9,458296542,False,14.0,Il mare era troppo salato e noi troppo graffiati.,ti amo.\n#MTVEMA where is The Weeknd????\nYOU ...,0,male
